In [1]:
import pandas as pd
import re
import requests

from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from urllib import parse

import time

In [3]:
def get_kko_category(url):
    # Selenium WebDriver 설정
    driver = webdriver.Chrome() 
    driver.get(url) 

    # 필요한 값이 로드될 때 까지 기다림
    try : 
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'swiper-slide.cont_slide.swiper-slide-active'))
        )
    except TimeoutException as e:
        driver.quit()
        return None


    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 값 가져오기 
    # 제품명 찾기
    product_name = soup.select_one('h2.tit_subject').get_text()

    # 검색 페이지 띄우기
    driver.get('https://gift.kakao.com/search/result?query=' + parse.quote(product_name) +'&searchType=typing_keyword') 

    # 카테고리 메뉴 클릭
    time.sleep(5)
    try:
        driver.find_element(By.XPATH, '//*[@id="mArticle"]/app-pw-result/div/div/app-search-result/app-option/div[3]/div/div/div/ul/li[2]/button').click()
    except:
        driver.find_element(By.XPATH, '//*[@id="mArticle"]/app-pw-result/div/div/app-search-result/app-option/div[2]/div/div/div/ul/li[2]/button').click()

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    category = soup.select_one('swiper-slide.list_slctcate.has_item_all.swiper-slide-active').select('li')[1].get_text().strip()

    
    # 브라우저 닫기
    driver.quit()
    return category

In [4]:
def get_29cm_category(url):
    # Selenium WebDriver 설정
    driver = webdriver.Chrome() 
    driver.get(url) 

    # 필요한 값이 로드될 때 까지 기다림
    try : 
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.css-70qvj9.e1mxp6e90'))
        )
    except TimeoutException as e:
        driver.quit()
        return None


    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 값 가져오기
    category_wrapper = soup.select('li.css-1vtc84p.e1mxp6e91')
    category = []
    for cat in category_wrapper:
        category.append(cat.select_one('span.css-96h8o6.e1w312mf1').get_text())

    category = '/'.join(category)
    
    # 브라우저 닫기
    driver.quit()
    return category

In [5]:
category_lst = []

for idx, url in enumerate(productDF['short_url']):
    if productDF['shoppingmall'][idx] == "카카오 선물하기":
        try: 
            category =  get_kko_category(url)
            category_lst.append(category)
        except:
            print(f"excepted : {idx}")
            category_lst.append(None)
    else:
        try: 
            category =  get_29cm_category(url)
            category_lst.append(category)
        except:
            print(f"excepted : {idx}")
            category_lst.append(None)
    time.sleep(3)



excepted : 16


In [7]:
productDF.to_csv('./productInfo/product(3).csv')

In [3]:
for_gpt_json = productDF[['product_name', 'short_url', 'original_price', 'brand_name','thumbnail']]
for_gpt_json.dropna(subset=['thumbnail'],inplace=True)
for_gpt_json.reset_index(inplace=True,drop=True)
for_gpt_json
for_gpt_json.to_json('./productInfo/product_forGPT.json', orient='index', force_ascii=False)

/var/folders/sp/rr963h912c1c9rvvd41sr4_h0000gn/T/ipykernel_1921/1477927711.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  for_gpt_json.dropna(subset=['thumbnail'],inplace=True)


In [2]:
productDF = pd.read_csv('./productInfo/product(3).csv', index_col=0)
productDF

,product_idx,video_id,product_id_inshop,product_url,shoppingmall,short_url,product_name,original_price,current_price,discount_rate,brand_name,brand_description,brand_url_inshop,product_fullinfo,review_num,review_sample,thumbnail,category
0,24,fflimV2pIL8,6252042,https://gift.kakao.com/product/6252042,카카오 선물하기,https://gift.kakao.com/product/6252042,[리본쇼핑백&헤어스크런치 증정] '팩과 클렌징을 한 번에' 휩드 비건 팩클렌저 1...,22700,22700,0,휩드,NaN,https://gift.kakao.com/brand/14844,"{'product_name': "" [리본쇼핑백&헤어스크런치 증정] '팩과 클렌징을 ...",756.0,"[{'star_rate': '별점 4점 만점 중 4점', 'created_at': ...",https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,스킨케어/메이크업
1,28,ONFJdkVFSzY,6784177,https://gift.kakao.com/product/6784177,카카오 선물하기,https://gift.kakao.com/product/6784177,월레스와 그로밋 일회용 필름 카메라 - 화려한 외출,27900,27900,0,콜리,NaN,https://gift.kakao.com/brand/14472,{'product_name': ' 월레스와 그로밋 일회용 필름 카메라 - 화려한 외...,36.0,"[{'star_rate': '별점 4점 만점 중 4점', 'created_at': ...",https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,카메라/캠코더/주변기기
2,29,ONFJdkVFSzY,5627132,https://gift.kakao.com/product/5627132,카카오 선물하기,https://gift.kakao.com/product/5627132,[단독+포장] 바디오일 & 샌달우드 괄사&에센셜오일 세트 / 3종 택1,29000,29000,0,톤28,NaN,https://gift.kakao.com/brand/10336,{'product_name': ' [단독+포장] 바디오일 & 샌달우드 괄사&에센셜오...,1306.0,"[{'star_rate': '별점 4점 만점 중 4점', 'created_at': ...",https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,향수/바디/헤어
3,31,kMHj6Zv4bW8,7695898,https://gift.kakao.com/product/7695898,카카오 선물하기,https://gift.kakao.com/product/7695898,"[단독/리본포장] ""푸른 텍스처+애프터배스 향"" 블루 웨이브 바디 오일 120ML",33000,33000,0,센녹,NaN,https://gift.kakao.com/brand/15661,"{'product_name': ' [단독/리본포장] ""푸른 텍스처+애프터배스 향"" ...",19.0,"[{'star_rate': '별점 4점 만점 중 4점', 'created_at': ...",https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,향수/바디/헤어
4,32,kMHj6Zv4bW8,7575783,https://gift.kakao.com/product/7575783,카카오 선물하기,https://gift.kakao.com/product/7575783,"[단독/선물포장] ""릴랙싱 리츄얼"" 안티 스트레스 밤+멀티 뿔 괄사 세트",37000,37000,0,센녹,NaN,https://gift.kakao.com/brand/15661,"{'product_name': ' [단독/선물포장] ""릴랙싱 리츄얼"" 안티 스트레스...",22.0,"[{'star_rate': '별점 4점 만점 중 4점', 'created_at': ...",https://img1.kakaocdn.net/thumb/C320x320@2x.fw...,향수/바디/헤어
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,57,YDgzXvhRkP4,1759079,https://product.29cm.co.kr/catalog/1759079?sou...,29cm,https://product.29cm.co.kr/catalog/1759079,오션 스트라이프 파자마 [여성] 3C,78000,54600,30,슬리피캣,"GOOD SLEEP, GOOD DAY.",https://shop.29cm.co.kr/brand/9809,"{'product_name': '오션 스트라이프 파자마 [여성] 3C', 'pric...",101.0,"[{'star_rate': 5, 'created_at': '2024.06.19', ...",https://img.29cm.co.kr/next-product/2022/10/05...,여성의류/홈웨어/세트
71,58,YDgzXvhRkP4,1759102,https://product.29cm.co.kr/catalog/1759102?sou...,29cm,https://product.29cm.co.kr/catalog/1759102,오션 스트라이프 파자마 [남성] 3C,78000,54600,30,슬리피캣,"GOOD SLEEP, GOOD DAY.",https://shop.29cm.co.kr/brand/9809,"{'product_name': '오션 스트라이프 파자마 [남성] 3C', 'pric...",24.0,"[{'star_rate': 5, 'created_at': '2024.06.16', ...",https://img.29cm.co.kr/next-product/2022/10/05...,남성의류/홈웨어/세트
72,60,YDgzXvhRkP4,1803839,https://product.29cm.co.kr/catalog/1803839?sou...,29cm,https://product.29cm.co.kr/catalog/1803839,TWIN MICRO SILVER,129000,110295,15,아호르,ahore는 A Harmony Of Rhythm & Essential의 약자로 흥미...,https://shop.29cm.co.kr/brand/16355,"{'product_name': 'TWIN MICRO SILVER', 'price':...",100.0,"[{'star_rate': 5, 'created_at': '2024.06.18', ...",https://img.29cm.co.kr/item/202403/11eeebe7a3e...,여성가방/크로스백
73,65,cMcpE-3re8c,874500,https://product.29cm.co.kr/catalog/874500,29cm,https://product.29cm.co.kr/catalog/874500,AT아이웨어머그오렌지10x11.5,77000,69300,10,셀레티,이탈리아만의 혁신적인 아이디어로 예술과 일상이 어우러진 라이프스타일 브랜드,https://shop.29cm.co.kr/brand/11621,"{'product_name': 'AT아이웨어머그오렌지10x11.5', 'price'...",6.0,"[{'star_rate': 5, 'created_at': '2022.07.23', ...",https://img.29cm.co.kr/next-product/2020/10/28...,"주방,생활/테이블웨어/커피잔,찻잔"
